In [1]:
%matplotlib inline
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import math

In [2]:
I = Image.open('Image_Data/i.png')
key1 = Image.open('Image_Data/key1.png')
key2 = Image.open('Image_Data/key2.png')
E = Image.open('Image_Data/E.png')
I = np.array(i)
key1 = np.array(k1)
key2 = np.array(k2)
E = np.array(e)
imshow(I,cmap=plt.cm.gray)

NameError: name 'i' is not defined